In [1]:
from tensorflow import keras
from tensorflow.keras.applications import resnet50
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.applications import mobilenet
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D

In [2]:
def resnet50_model():
    model_input_shape = (224,224)
    model = resnet50.ResNet50(include_top=True, weights='imagenet', input_shape=(224, 224, 3), classes=1000)
    model = keras.Model(inputs=model.input, outputs=model.get_layer(index=-2).output)
    return (model, model_input_shape)

In [3]:
def inceptionv3_model():
    model_input_shape = (299, 299) 
    model = inception_v3.InceptionV3(include_top=True, weights='imagenet', input_shape=(299, 299, 3), classes=1000)
    model = keras.Model(inputs=model.input, outputs=model.get_layer(index=-2).output)
    return (model, model_input_shape)

In [1]:
def mobilenet_model():
    model_input_shape = (224, 224) 
    model = mobilenet.MobileNet(include_top=True, weights='imagenet', input_shape=(224, 224, 3), classes=1000)
    model = keras.Model(inputs=model.input, outputs=model.get_layer(index=-2).output)
    return (model, model_input_shape)

In [11]:
def chexnet_model():
    #chexnet_weights_path = "../input/chexnet-keras-weights/brucechou1983_CheXNet_Keras_0.3.0_weights.h5"
    chexnet_weights_path = "./brucechou1983_CheXNet_Keras_0.3.0_weights.h5"
    IMG_SIZE = 512
    model_input_shape = (IMG_SIZE, IMG_SIZE) 
    base = DenseNet121(weights=None,
                            include_top=False,
                            input_shape=(IMG_SIZE,IMG_SIZE,3)
                           )
    ## workaround - add dummy layer then load weights then pop dummy layer, in order to match expected shape for pretrained weights
    predictions = keras.layers.Dense(14, activation='sigmoid', name='predictions')(base.output)
    ## ,by_name=True - could save on workaround, but don't know if names will necessarily match + how to validate? - https://github.com/keras-team/keras/issues/5397
    base = keras.Model(inputs=base.input, outputs=predictions) 
    base.load_weights(chexnet_weights_path)
    base.trainable=False # freeze most layers
    base.training=False

    base.layers.pop()

    ### https://stackoverflow.com/questions/41668813/how-to-add-and-remove-new-layers-in-keras-after-loading-weights
    new_model = GlobalAveragePooling2D()(base.layers[-4].output) 

    model = keras.Model(base.input, new_model)
    return (model, model_input_shape)